In [1]:
import re
import gensim
import string
import warnings
from collections import Counter
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

morph = MorphAnalyzer()
warnings.filterwarnings('ignore')

# Данные

In [2]:
# Дополним словарь стоп-слов синтаксисом Python, цифрами, двумя тире, рандомными словами.
stops = set(stopwords.words('russian')) | {'gt', 'from', 'import', 'and', 'or', 'is', 'in',
                                           'if', 'for', 'while', 'none', 'null', 'return', 'yield',
                                           'break', 'pass', 'continue', 'int'} | \
                                            set(map(str, range(10))) | \
                                            {'–', '—', 'очень', 'n', 'a', 'x'}

def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, top=50000):
    unique = Counter()
    for text in texts:
        unique.update(text)
    
    norm_unique = {word: morph.parse(word)[0].normal_form for word, _ in unique.most_common(top)}
    
    norm_texts = []
    for text in texts:
        norm_words = [norm_unique.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    # Для токенизации используем word_tokenize из nltk.
    words = [word.strip(string.punctuation) for word in word_tokenize(text)]
    words = [word for word in words if word]
    
    return words

In [3]:
texts = open('habr_texts.txt').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts])

In [4]:
texts[0][:25]

['masstransit',
 'это',
 'open',
 'source',
 'библиотека',
 'разработать',
 'язык',
 'c',
 'net',
 'платформа',
 'работа',
 'шина',
 'дать',
 'который',
 'использоваться',
 'построение',
 'распределенный',
 'приложение',
 'реализация',
 'soa',
 'service',
 'oriented',
 'architecture',
 'качество',
 'message']

# Энграммы

In [5]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.3)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [6]:
ngrammed_texts[0][:25]

['masstransit',
 'это',
 'open_source',
 'библиотека',
 'разработать',
 'язык_c',
 'net',
 'платформа',
 'работа',
 'шина',
 'дать',
 'который',
 'использоваться',
 'построение_распределенный',
 'приложение',
 'реализация',
 'soa',
 'service',
 'oriented',
 'architecture',
 'качество',
 'message',
 'мочь',
 'выступать',
 'rabbitmq']

Появились осмысленные энграммы: "open\_source", "язык\_C", "построение\_распределенный".

# LDA

In [7]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

# Будем не так строго фильтровать частые слова и более строго - редкие.
dictionary.filter_extremes(no_above=0.5, no_below=40)
dictionary.compactify()

In [8]:
# Мы отфильтровали стоп-слова.

assert all(stop not in dictionary.token2id for stop in stops)

In [9]:
print(dictionary)

Dictionary(5739 unique tokens: ['2-х', '3.0', 'address', 'api', 'azure']...)


In [10]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

In [11]:
lda = gensim.models.LdaMulticore(corpus, num_topics=100, id2word=dictionary, passes=10)
topics = lda.print_topics()

Полученная модель оставляет желать лучшего: только некоторые темы хорошо интерпретируемы.

In [13]:
# ООП.

topics[3]

(81,
 '0.021*"функция" + 0.018*"объект" + 0.013*"значение" + 0.012*"код" + 0.010*"класс" + 0.009*"метод" + 0.008*"тип" + 0.008*"реализация" + 0.007*"память" + 0.007*"результат"')

In [15]:
# Космос.

topics[7]

(91,
 '0.011*"звезда" + 0.010*"год" + 0.010*"галактика" + 0.010*"система" + 0.008*"метр" + 0.008*"объект" + 0.007*"вселенная" + 0.006*"здание" + 0.006*"земля" + 0.006*"километр"')

In [39]:
# Снова космос.

topics[11]

(11,
 '0.012*"год" + 0.011*"«_»" + 0.007*"система" + 0.007*"земля" + 0.006*"спутник" + 0.006*"проект" + 0.005*"аппарат" + 0.004*"звезда" + 0.004*"станция" + 0.004*"энергия"')

In [40]:
# Типизация и функциональное программирование.

topics[12]

(12,
 '0.016*"функция" + 0.012*"код" + 0.009*"значение" + 0.007*"память" + 0.006*"программа" + 0.006*"число" + 0.006*"тип" + 0.006*"•" + 0.005*"массив" + 0.005*"результат"')

In [16]:
lda.log_perplexity(corpus[:10000])

-7.925039593676748

# TF-IDF

In [17]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
tfidf_corpus = tfidf[corpus]

In [18]:
tfidf_lda = gensim.models.LdaMulticore(tfidf_corpus, num_topics=100, id2word=dictionary, passes=10)
tfidf_topics = tfidf_lda.print_topics()

In [20]:
# Мессенджеры, социальные сети, корпоративные темы + разный мусор.

tfidf_topics[3]

(45,
 '0.001*"метро" + 0.001*"mail.ru" + 0.001*"мессенджер" + 0.001*"приложение" + 0.001*"сотрудничество" + 0.001*"социальный_сеть" + 0.001*"миссия" + 0.001*"виджет" + 0.000*"генеральный_директор" + 0.000*"вызов"')

In [51]:
# Космос + разный мусор.

tfidf_topics[9]

(24,
 '0.016*"станция" + 0.007*"иначе" + 0.007*"собеседник" + 0.002*"космический" + 0.002*"патч" + 0.001*"’" + 0.001*"антенна" + 0.001*"ракета" + 0.001*"строительство" + 0.001*"сигнал"')

In [53]:
# Генетика + разный мусор.

tfidf_topics[11]

(82,
 '0.015*"удача" + 0.009*"клетка" + 0.008*"женщина" + 0.007*"телевизор" + 0.007*"пациент" + 0.007*"замедление" + 0.005*"слишком_сложный" + 0.005*"днк" + 0.004*"ген" + 0.004*"61"')

In [23]:
tfidf_lda.log_perplexity(tfidf_corpus[:10000])

-19.252105176212897

Увеличение значения перплексии по модулю согласуется с визуальным ощущением от просмотра тем: они стали ещё хуже. Во многих случаях очевидно смешение нескольких тем в одной, во всех темах есть мусор. Сравнивать темы по отдельности трудно, поскольку нужно установить "выравнивание" между двумя наборами по 100 тем, а главное, этого выравнивания, скорее всего, и нет.

# NMF

In [24]:
stexts = [' '.join(text) for text in ngrammed_texts]
vectorizer = TfidfVectorizer(max_features=1000, min_df=40, max_df=0.5, ngram_range=(1, 3))
X = vectorizer.fit_transform(stexts)

In [25]:
model = NMF(n_components=20)
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=20, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [26]:
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:, :-11:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i+1, words)

1 ['делать', 'какой', 'что', 'какой то', 'ваш', 'книга', 'человек', 'что то', 'хотеть', 'знать']
2 ['функция', 'код', 'объект', 'значение', 'метод', 'класс', 'тип', 'алгоритм', 'строка', 'элемент']
3 ['игра', 'игрок', 'игровой', 'играть', 'разработчик', 'уровень', 'разработка', 'мир', 'карта', 'экран']
4 ['приложение', 'пользователь', 'android', 'разработчик', 'мобильный', 'разработка', 'сервис', 'api', 'google', 'платформа']
5 ['устройство', 'смартфон', 'датчик', 'функция', 'атака', 'производитель', 'безопасность', 'android', 'защита', 'драйвер']
6 ['сервер', 'клиент', 'запрос', 'сервис', 'дата центр', 'дата', 'центр', 'база_дать', 'облако', 'инфраструктура']
7 ['компания', 'сотрудник', 'год', 'рынок', 'бизнес', 'клиент', 'google', 'российский', 'microsoft', 'продажа']
8 ['камера', 'смартфон', 'звук', 'цена', 'экран', 'видео', 'модель', 'телефон', 'корпус', 'процессор']
9 ['файл', 'скрипт', 'настройка', 'пакет', 'папка', 'команда', 'строка', 'добавить', 'репозиторий', 'установка']
10 

Почти все темы кажутся удачными и интерпретируемыми.
К неудачным можно отнести только темы 1, 13, 15 и 20. Также некоторые темы пересекаются.

1: ?философия и литература
2: ООП
3: разработка игр
4: веб-разработка
5: гаджеты
6: инфраструктура данных
7: IT-бизнес
8: снова гаджеты
9: файлы, скрипты, репозитории
10: когнитивные исследования
11: веб
12: космос
13: какие-то языки программирования
14: управление IT-командой
15: снова какие-то языки программирования
16: технологии производства
17: доклады и конференции
18: сеть
19: система
20: ?frontend, веб

# LDA с 20 темами

In [27]:
lda = gensim.models.LdaMulticore(corpus, num_topics=20, id2word=dictionary, passes=10)
topics = lda.print_topics()

In [28]:
for i, topic in topics:
    print(i+1, re.findall(r'(?<=")[A-Za-zА-Яа-яЁё]+(?=")', topic))

1 ['человек', 'компания', 'год', 'делать', 'проект', 'говорить', 'деньга', 'сотрудник', 'хотеть']
2 ['lt', 'name', 'string', 'the', 'i', 'this', 'amp', 'файл', 'type']
3 ['запрос', 'сервер', 'система', 'диск', 'пользователь', 'проблема', 'тест', 'запись']
4 ['код', 'класс', 'объект', 'метод', 'файл', 'элемент', 'тип', 'функция', 'приложение', 'пример']
5 ['человек', 'год', 'компания', 'робот', 'система', 'машина', 'технология', 'автомобиль', 'ия', 'компьютер']
6 ['звук', 'человек', 'глаз', 'учёный', 'движение', 'тело', 'пациент', 'случай', 'система']
7 ['проект', 'задача', 'разработка', 'разработчик', 'сайт', 'делать', 'продукт', 'игра', 'команда', 'язык']
8 ['человек', 'год', 'ребёнок', 'ваш', 'исследование', 'мозг', 'курс', 'результат']
9 ['точка', 'объект', 'значение', 'алгоритм', 'вселенная', 'тип', 'число', 'результат', 'количество', 'координата']
10 ['игра', 'игрок', 'игровой', 'играть', 'процессор', 'сервер', 'система', 'уровень', 'видеокарта']
11 ['компания', 'пользователь', 'с

Видно, что в LDA почти все темы получились грязными (смесь нескольких тем) либо совершенно неинтерпретируемыми.